# Sampling

we sample from chemical graphs.

First we will fit a sampler and then we will generate new graphs :)


## initialising logging and notebook

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=2)
!date
BABELDRAW=False

Mon May 23 14:19:38 CEST 2016


## fit sampler i.e. fit estimator and grammar

In [2]:
# get data
from eden.converter.graph.gspan import gspan_to_eden
from itertools import islice
def get_graphs(dataset_fname, size=100):
    return  islice(gspan_to_eden(dataset_fname),size)

dataset_fname = 'toolsdata/bursi.pos.gspan'

In [3]:
%%time
'''
TRAINING A SAMPLER
'''
from graphlearn.graphlearn import  Sampler
from eden.graph import Vectorizer

# to demonstrate pluggable regressor
#from graphlearn.estimate import OneClassEstimator
#from sklearn.linear_model import LinearRegression 

# get training graphs
training_graphs = get_graphs(dataset_fname, size=300)

# train a sampler
sampler=Sampler(radius_list=[0,1], thickness_list=[2],random_state=42447,
                          min_cip_count=2, min_interface_count=2,
                          vectorizer=Vectorizer(3)
                          #estimator = OneClassEstimator(classifier=LinearRegression()))
                           )
sampler.fit(training_graphs)

# lets look at a few stats about the trained sampler
print('graph grammar stats:')
n_instances, interface_counts, core_counts, cip_counts = sampler.grammar().size()
print('#instances: %d   #interfaces: %d   #cores: %d   #core-interface-pairs: %d' % (n_instances, interface_counts, core_counts, cip_counts))
sampler.save('tmp/sampler.ge')


Classifier:
SGDClassifier(alpha=0.000545041563449, average=False, class_weight=None,
       epsilon=0.1, eta0=0.0356642960812, fit_intercept=True,
       l1_ratio=0.15, learning_rate='invscaling', loss='log', n_iter=56,
       n_jobs=1, penalty='l1', power_t=1.07204913259, random_state=None,
       shuffle=True, verbose=0, warm_start=False)

Predictive performance:
            accuracy: 1.000 +- 0.000
           precision: 1.000 +- 0.000
              recall: 1.000 +- 0.000
                  f1: 1.000 +- 0.000
   average_precision: 1.000 +- 0.000
             roc_auc: 1.000 +- 0.000
#instances: 300  #interfaces: 122   #cores: 54   #core-interface-pairs: 378
graph grammar stats:
#instances: 300   #interfaces: 122   #cores: 54   #core-interface-pairs: 378
Saved model: tmp/sampler.ge
CPU times: user 15 s, sys: 753 ms, total: 15.8 s
Wall time: 40.2 s


# Sample and show generated graphs

In [4]:
%%time
from itertools import islice

"""
USING A SAMPLER
"""
# reloading, this is not necessary actually since we sould still have the object in memory 
from graphlearn.graphlearn import  Sampler
sampler=Sampler()
sampler.load('tmp/sampler.ge')


# get a graph list and pic some graphs to initialize the sampler with.
# the sampler will look at each graphs and try n_steps times to alter it.
graphs = get_graphs(dataset_fname, size=100)
id_start=34
id_end=id_start+12
input_graphs = islice(graphs,id_start,id_end)

# sample parameters
n_steps=60 # how many steps
n_samples=4 # collect this many samples during the process


# sampler will return lists of graphs as you see below.
graphs = sampler.sample(input_graphs,
                        n_steps=n_steps, n_samples=n_samples,
                        target_orig_cip=False,
                        probabilistic_core_choice=False,
                        score_core_choice= True,
                        max_size_diff=1,
                        burnin=10,
                        include_seed=True,
                        proposal_probability = False,
                        improving_threshold=.5, 
                        improving_linear_start=0.0,
                        accept_static_penalty=0.0,
                        n_jobs=1,
                        select_cip_max_tries=200,
                        keep_duplicates=True,  
                        monitor=True)

Loaded model: tmp/sampler.ge


TypeError: sample() got an unexpected keyword argument 'target_orig_cip'

In [ ]:
%%time
# plot examples of sampling paths

scores=[]
ids=range(id_start,id_end)
for i,graphlist in enumerate(graphs):
    print 'Graph id: %d'%(ids[i])
    scores.append(sampler.monitors[i].sampling_info['score_history'])
    if BABELDRAW:
        from graphlearn.utils import openbabel
        openbabel.draw(graphlist, d3=False, n_graphs_per_line=6,size=200)
    else:
        from graphlearn.utils import draw
        draw.graphlearn(graphlist,contract=True,#vertex_label='id',
                   n_graphs_per_line=6, size=10, 
                   colormap='Paired', invert_colormap=False,node_border=0.5, vertex_color='_labels_',
                   vertex_alpha=0.5, edge_alpha=0.2, node_size=450)
    
 
    

In [ ]:
# we sampled with monitoring mode enabled, 
#this is why we could now look at what exactly happened during sampling
#this is step 9 of the 1st graph
sampler.monitors[1][9]

# Show sample score history

In [ ]:
%%time
# plot sampling path score
from itertools import islice
import numpy as np
import pylab as plt
markevery=n_steps/(n_samples)
step=1
num_graphs_per_plot=3
num_plots=np.ceil([len(scores)/num_graphs_per_plot])
for i in range(num_plots):

    plt.figure(figsize=(13,5))
    for j,score in enumerate(scores[i*num_graphs_per_plot:i*num_graphs_per_plot+num_graphs_per_plot]):
     
        data = list(islice(score,None, None, step))
        plt.plot(data, linewidth=2, label='graph %d'%(j+i*num_graphs_per_plot+id_start))
        plt.plot(data, linestyle='None',markevery=markevery, markerfacecolor='white', marker='o', markeredgewidth=2,markersize=6)
    plt.legend(loc='lower right')
    plt.grid()
    plt.xlim(-1,n_steps+1)
    plt.ylim(-0.1,1.1)
    plt.show()

---

In [ ]:
from matplotlib.cm import inferno
colors = inferno(range(256))
print colors
import matplotlib
for x in colors:
    print matplotlib.colors.rgb2hex(x)

In [ ]:
z=Namespace(decomposergen='asd', estimator=4)